In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'smiling-audio-287409-77f38c77b232.json'
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [3]:
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [4]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


In [5]:
table_ref = dataset_ref.table("comments")
table = client.get_table(table_ref)

In [6]:
client.list_rows(table,max_results=5).to_dataframe()

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [7]:
#figureout the no of comments that were made as responses to a specific comment

#-> the [parent]  col indicates the comment was replied to
#-> the [id]  col has uniqueid used to identify each comments

#also look at popular comments ie more than 10 replies

query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id)>10

"""

In [8]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_popular,job_config=safe_config)

popular_comments = query_job.to_dataframe()

popular_comments.head()

,parent,f0_
0,6427895,46
1,202918,44
2,8120079,148
3,9016949,38
4,7075537,51


__ALIASING and OTHER IMPROVEMENTS__
 
-> the col resulting from COUNT(id) was called f0_, which is not very descriptive.
--> we can change the name by adding __AS NumPosts__ after you specify the aggregation, This is called __Aliasing__

-> We can also use __COUNT(1)__ instead of __COUNT("name of col")__ to count rows in each group. It makes query faster and use less data.




In [9]:
query_improved = """
                SELECT parent, COUNT(id) as NumPosts
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id)>10

"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_improved,job_config=safe_config)

improved_popular_comments = query_job.to_dataframe()

improved_popular_comments.head()

,parent,NumPosts
0,3873271,50
1,5308611,63
2,5176140,63
3,9269660,51
4,9439286,69


__If you have a GROUP BY clause, then all the var must be passed to either a-__

-> GROUP BY command or
-> an aggregate funtion like COUNT()

In [10]:
query_bad = """  
            SELECT author, parent, count(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
"""

#### Here [author] col is not passed to an aggregate fun or a GROUP BY clause.


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_bad
                         ,job_config=safe_config)

bad = query_job.to_dataframe()
bad.head()

BadRequest: 400 SELECT list expression references column author which is neither grouped nor aggregated at [2:20]

(job ID: 4cae10bc-5d35-4e48-8163-945cae8abd80)

                 -----Query Job SQL Follows-----                 

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:  
   2:            SELECT author, parent, count(id)
   3:            FROM `bigquery-public-data.hacker_news.comments`
   4:            GROUP BY parent
    |    .    |    .    |    .    |    .    |    .    |    .    |

### ERROR because of author col-> SELECT list expression references column author which is neither grouped nor aggregated at

In [12]:
prolific_commenters_query = """

                SELECT author, COUNT(1) AS NumPosts
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY author
                HAVING COUNT(1) > 10000
"""           

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

         author  NumPosts
0  dragonwriter     10723
1          None    227736
2         DanBC     12902
3         sp332     10882
4        davidw     10764


In [13]:
deleted_query = """
             SELECT COUNT(1) AS num_deleted_posts
             FROM `bigquery-public-data.hacker_news.comments`
             WHERE deleted = TRUE
"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10 ** 10)
query_job  =  client.query(deleted_query, job_config= safe_config)

deleted_comments = query_job.to_dataframe()

deleted_comments.head()

,num_deleted_posts
0,227736
